# Testlauf für 4 Felder

### Wir nehmen an, dass unser Landwirt 4 Felder hat für welche er unser Tool nutzen möchte

### Als erstes müssen die Daten des Feldes (Bodendaten + Größe + Koordinaten) spezifiziert werden

In [1]:
feldliste = [
             [10, 59, 15, 29.83040388, 89.30428305, 6.32400451, 58.86687093, 3000, 57.4778, 4.2269],
             [13, 144, 204, 30.7280404, 82.42614055, 6.092241627, 68.38135469, 12000, 63.4305, 10.3951], 
             [98, 27, 27, 24.71384065, 51.29142534, 7.238109556, 197.6439711, 17000, 46.0569, 14.5058], 
             [101, 20, 48, 24.6774157, 82.75411437, 6.206247494, 57.05709413, 7000, 41.3954, 2.1601], 
             [9, 143, 197, 23.75033085, 92.88160462, 5.570020684, 117.6602827, 1500, 49.5766, 11.0426],
             [34, 10, 14, 34.05296914, 92.05811721, 6.725600855, 116.8020848, 30000, 43.2259, 23.9623],
             [127, 53, 24, 22.21506982, 76.17851932, 6.127939628, 70.40557612, 10000, 54.3242, 3.0573],
             [59, 60, 84, 19.23025305, 18.96725565, 7.690962338, 95.70992037, 14500, 43.9106, 12.9132]
            ]

### Trainieren des ML Algorithmus

In [2]:
# Imports
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Laden der Daten aus der CSV-Datei
data = pd.read_csv('crop_recommendation.csv')

# Aufteilen der Daten in Features und Labels
X = data.drop('label', axis=1)
y = data['label']

# Teilen der Daten in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Erstellen eines DescisionTree-Classifiers
clf = DecisionTreeClassifier()

# Trainieren des Algorithmus auf den Trainingsdaten
clf.fit(X_train, y_train)

# Vorhersagen mit den Testdaten
y_pred = clf.predict(X_test)

# Auswertung der Leistung des Algorithmus
genauigkeit = accuracy_score(y_test, y_pred)

genauigkeit

1.0

In [3]:
feldliste

[[10,
  59,
  15,
  29.83040388,
  89.30428305,
  6.32400451,
  58.86687093,
  3000,
  57.4778,
  4.2269],
 [13,
  144,
  204,
  30.7280404,
  82.42614055,
  6.092241627,
  68.38135469,
  12000,
  63.4305,
  10.3951],
 [98,
  27,
  27,
  24.71384065,
  51.29142534,
  7.238109556,
  197.6439711,
  17000,
  46.0569,
  14.5058],
 [101,
  20,
  48,
  24.6774157,
  82.75411437,
  6.206247494,
  57.05709413,
  7000,
  41.3954,
  2.1601],
 [9,
  143,
  197,
  23.75033085,
  92.88160462,
  5.570020684,
  117.6602827,
  1500,
  49.5766,
  11.0426],
 [34,
  10,
  14,
  34.05296914,
  92.05811721,
  6.725600855,
  116.8020848,
  30000,
  43.2259,
  23.9623],
 [127,
  53,
  24,
  22.21506982,
  76.17851932,
  6.127939628,
  70.40557612,
  10000,
  54.3242,
  3.0573],
 [59,
  60,
  84,
  19.23025305,
  18.96725565,
  7.690962338,
  95.70992037,
  14500,
  43.9106,
  12.9132]]

In [4]:
predict_list = []

for f in feldliste:
    predict_list.append(f[:-3])
    
predict_df = pd.DataFrame(predict_list, columns=['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall'])

felder_predict = clf.predict(predict_df)
felder_predict

array(['cowpeas', 'grapes', 'coffee', 'watermelon', 'apple', 'orange',
       'cotton', 'Soyabeans'], dtype=object)

In [5]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()
username = os.environ.get('AZURE_USER')
password = os.environ.get('PASSWORD')
host = os.environ.get('HOST')
db = os.environ.get('DB')

# Establish the connection
conn = mysql.connector.connect(
  user=username, 
  password=password, 
  host=host, 
  database=db
)
print(conn)

In [6]:
cursor = conn.cursor()

In [7]:
insert_list = []
for idx, item in enumerate(feldliste):
    insert_list.append(item[-3:])
    insert_list[idx].append(felder_predict[idx])
    
insert_list

[[3000, 57.4778, 4.2269, 'cowpeas'],
 [12000, 63.4305, 10.3951, 'grapes'],
 [17000, 46.0569, 14.5058, 'coffee'],
 [7000, 41.3954, 2.1601, 'watermelon'],
 [1500, 49.5766, 11.0426, 'apple'],
 [30000, 43.2259, 23.9623, 'orange'],
 [10000, 54.3242, 3.0573, 'cotton'],
 [14500, 43.9106, 12.9132, 'Soyabeans']]

In [8]:
for idx, item in enumerate(insert_list):   
    cursor.execute(f"SELECT * FROM ernte WHERE ernte_name='{item[3]}'")
    result = cursor.fetchall()
    item[3] = result[0][0]
    insert_list[idx] = tuple(item)
    
sql_code = "INSERT INTO feld (feld_groesse, lat, lon, ernte_id) VALUES (%s, %s, %s, %s)"

cursor.executemany(sql_code, insert_list)
conn.commit()

In [9]:
import requests
from datetime import datetime

api_key = os.environ.get('API_KEY')
cursor.execute("SELECT * FROM feld")
feld_query = cursor.fetchall()


for feld in feld_query:
    response = requests.get(f"https://api.openweathermap.org/data/3.0/onecall?lat={feld[2]}&lon={feld[3]}&exclude=current,minutely,hourly,alerts&appid={api_key}")
    
    rain_insert_list = []
    
    for day in response.json()["daily"]:
        date_value = datetime.fromtimestamp(day['dt']).strftime('%Y-%m-%d')
        
        if "rain" in day:
            rain_value = day['rain']
        else:
            rain_value = 0.0
            
        feld_value = feld[0]
        
        rain_insert_values = (date_value, rain_value, feld_value)
        
        rain_insert_list.append(rain_insert_values)

    
    sql_code = "INSERT INTO niederschlag_pro_tag (datum, niederschlag, feld_id) VALUES (%s, %s, %s)"

    cursor.executemany(sql_code, rain_insert_list)
    conn.commit()
    

In [10]:
cursor.execute("SELECT * FROM niederschlag_pro_tag")
rain_query = cursor.fetchall()

rain_query

[('2023-04-05', 0.0, 1),
 ('2023-04-06', 0.11, 1),
 ('2023-04-07', 0.0, 1),
 ('2023-04-08', 0.0, 1),
 ('2023-04-09', 0.13, 1),
 ('2023-04-10', 1.39, 1),
 ('2023-04-11', 6.29, 1),
 ('2023-04-12', 3.2, 1),
 ('2023-04-06', 0.0, 2),
 ('2023-04-07', 0.0, 2),
 ('2023-04-08', 0.0, 2),
 ('2023-04-09', 0.0, 2),
 ('2023-04-10', 0.0, 2),
 ('2023-04-11', 0.0, 2),
 ('2023-04-12', 0.0, 2),
 ('2023-04-13', 0.0, 2),
 ('2023-04-06', 0.0, 3),
 ('2023-04-07', 0.0, 3),
 ('2023-04-08', 1.62, 3),
 ('2023-04-09', 1.47, 3),
 ('2023-04-10', 0.0, 3),
 ('2023-04-11', 7.6, 3),
 ('2023-04-12', 7.78, 3),
 ('2023-04-13', 1.76, 3),
 ('2023-04-06', 0.0, 4),
 ('2023-04-07', 0.0, 4),
 ('2023-04-08', 0.0, 4),
 ('2023-04-09', 0.0, 4),
 ('2023-04-10', 0.0, 4),
 ('2023-04-11', 0.0, 4),
 ('2023-04-12', 0.0, 4),
 ('2023-04-13', 4.46, 4),
 ('2023-04-06', 0.0, 5),
 ('2023-04-07', 0.0, 5),
 ('2023-04-08', 0.0, 5),
 ('2023-04-09', 0.0, 5),
 ('2023-04-10', 0.35, 5),
 ('2023-04-11', 6.91, 5),
 ('2023-04-12', 1.17, 5),
 ('2023-04-13

In [11]:
cursor.execute("SELECT * FROM feld")
feld_query = cursor.fetchall()
final_value_insert = []

for feld in feld_query:
    feld_id = feld[0]
    feld_groesse = feld[1]
    
    cursor.execute(f"SELECT * FROM ernte WHERE ernte_id='{feld[-1]}'")
    ernte_query = cursor.fetchall()
    
    ernte_id = ernte_query[0][0]
    wasser_konsum = ernte_query[0][2]
    
    cursor.execute(f"SELECT * FROM niederschlag_pro_tag WHERE feld_id='{feld_id}'")
    rain_query = cursor.fetchall()
    
    for rain_value in rain_query:
        wasserbedarf_insert_value = (wasser_konsum - rain_value[1])
        
        if wasserbedarf_insert_value < 0:
            wasserbedarf_insert_value = 0
        
        wasserbedarf_insert_value_combined = (wasser_konsum - rain_value[1]) * feld_groesse
        
        if wasserbedarf_insert_value_combined < 0:
            wasserbedarf_insert_value_combined = 0
            
        insert_tuple = (feld_id, ernte_id, rain_value[0], wasserbedarf_insert_value_combined, wasserbedarf_insert_value)
        final_value_insert.append(insert_tuple)

        
sql_code = "INSERT INTO wasserbedarfsanalyse (feld_id, ernte_id, datum, wasserbedarf_pro_tag, wasserbedarf_pro_tag_pro_m2) VALUES (%s, %s, %s, %s, %s)"

cursor.executemany(sql_code, final_value_insert)
conn.commit()
    

In [12]:
cursor.execute("SELECT * FROM wasserbedarfsanalyse")
wasserbedarf_query = cursor.fetchall()
wasserbedarf_query

[(1, 7, '2023-04-05', 18000.0, 6.0),
 (1, 7, '2023-04-06', 17670.0, 5.89),
 (1, 7, '2023-04-07', 18000.0, 6.0),
 (1, 7, '2023-04-08', 18000.0, 6.0),
 (1, 7, '2023-04-09', 17610.0, 5.87),
 (1, 7, '2023-04-10', 13830.0, 4.61),
 (1, 7, '2023-04-11', 0.0, 0.0),
 (1, 7, '2023-04-12', 8400.0, 2.8),
 (2, 10, '2023-04-06', 96000.0, 8.0),
 (2, 10, '2023-04-07', 96000.0, 8.0),
 (2, 10, '2023-04-08', 96000.0, 8.0),
 (2, 10, '2023-04-09', 96000.0, 8.0),
 (2, 10, '2023-04-10', 96000.0, 8.0),
 (2, 10, '2023-04-11', 96000.0, 8.0),
 (2, 10, '2023-04-12', 96000.0, 8.0),
 (2, 10, '2023-04-13', 96000.0, 8.0),
 (3, 15, '2023-04-06', 102000.0, 6.0),
 (3, 15, '2023-04-07', 102000.0, 6.0),
 (3, 15, '2023-04-08', 74460.0, 4.38),
 (3, 15, '2023-04-09', 77010.0, 4.53),
 (3, 15, '2023-04-10', 102000.0, 6.0),
 (3, 15, '2023-04-11', 0.0, 0.0),
 (3, 15, '2023-04-12', 0.0, 0.0),
 (3, 15, '2023-04-13', 72080.0, 4.24),
 (4, 11, '2023-04-06', 49000.0, 7.0),
 (4, 11, '2023-04-07', 49000.0, 7.0),
 (4, 11, '2023-04-08', 4